In [ ]:
# ==========================
# 1. Imports y Configuración
# ==========================

import sys
import random
import json
from pathlib import Path

# Ajustar la ruta para que Python reconozca 'src'
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import MIDI_DIR, WAV_DIR, JAMS_DIR, DEFAULT_TEMPO
from src.generate_progression import generate_progression
from src.audio_conversion import convert_all_mid_in_folder
from src.jams_creation import create_jams_file

print("MIDI_DIR =", MIDI_DIR)
print("WAV_DIR  =", WAV_DIR)
print("JAMS_DIR =", JAMS_DIR)
print("DEFAULT_TEMPO =", DEFAULT_TEMPO)


In [ ]:
# 2. Generar progresiones .mid
prog_all_1 = "I-IV-V"
name_all_1 = prog_all_1

generate_progression(prog_all_1, name_all_1, MIDI_DIR, DEFAULT_TEMPO)

midi_folder_all_1 = MIDI_DIR / name_all_1

In [ ]:
# 3. Crear .jams parseando nombre del .mid y leyendo durations.json

def strip_suffix(token: str) -> str:
    if "_" in token:
        return token.split("_", 1)[0]
    return token

def parse_filename_and_create_jams(midi_file: Path, durations_dict: dict):
    base_name = midi_file.stem
    parts = base_name.split("-")
    if len(parts) < 3:
        print(f"Nombre de archivo inesperado => {base_name}")
        return

    note_plus_oct = "-".join(parts[:2])
    progression_list = parts[2:-1]
    raw_tokens = "-".join(progression_list).split("-")
    roman_sequence = [strip_suffix(tok) for tok in raw_tokens]

    splitted = note_plus_oct.split("-")
    tonalidad = strip_suffix(splitted[0]) if len(splitted) >= 1 else "C"

    durations = durations_dict.get(f"{base_name}.mid", None)

    jam_path = create_jams_file(
        roman_sequence=roman_sequence,
        key=tonalidad,
        jam_name=base_name,
        progression_name="-".join(roman_sequence),
        durations=durations
    )
    print(f"[JAMS] {midi_file.name} => {jam_path.name}")

def parse_and_create_jams_in_folder(folder: Path):
    if not folder.exists():
        print(f"No existe carpeta => {folder}")
        return

    mid_files = list(folder.rglob("*.mid"))
    if not mid_files:
        print(f"No hay .mid en => {folder}")
        return

    durations_path = folder / "durations.json"
    if durations_path.exists():
        with open(durations_path, "r") as f:
            durations_dict = json.load(f)
    else:
        durations_dict = {}

    for mf in mid_files:
        parse_filename_and_create_jams(mf, durations_dict)

In [ ]:
# Ejecutar creación de .jams
parse_and_create_jams_in_folder(midi_folder_all_1)
